In [ ]:
# Install opencv
# !pip install opencv-python

In [1]:
# Import libraries

import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from scipy import ndimage as ndi
import pandas as pd
import json
import os
import timeit
import random

In [2]:
import os
import pandas as pd

# Read files in folder
def get_subfiles(dir):
    "Get a list of immediate subfiles"
    return next(os.walk(dir))[2]

In [3]:
# Show Images function
def ShowImage(ImageList, nRows = 1, nCols = 2, WidthSpace = 0.00, HeightSpace = 0.00):
    from matplotlib import pyplot as plt
    import matplotlib.gridspec as gridspec

    gs = gridspec.GridSpec(nRows, nCols)
    gs.update(wspace=WidthSpace, hspace=HeightSpace) # set the spacing between axes.
    plt.figure(figsize=(20,20))
    for i in range(len(ImageList)):
        ax1 = plt.subplot(gs[i])
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        ax1.set_aspect('equal')
        plt.subplot(nRows, nCols,i+1)
        image = ImageList[i].copy()
        if (len(image.shape) < 3):
            plt.imshow(image, plt.cm.gray)
        else:
            plt.imshow(image)
        plt.title("Image " + str(i))
        plt.axis('off')
    plt.show()

In [4]:
# Resize image function
def ResizeImage(IM, DesiredWidth, DesiredHeight):
    from skimage.transform import rescale, resize

    OrigWidth = float(IM.shape[1])
    OrigHeight = float(IM.shape[0])
    Width = DesiredWidth
    Height = DesiredHeight

    if ((Width == 0) & (Height == 0)):
        return IM

    if (Width == 0):
        Width = int((OrigWidth * Height)/OrigHeight)

    if (Height == 0):
        Height = int((OrigHeight * Width)/OrigWidth)

    dim = (Width, Height)
    resizedIM = cv2.resize(IM, dim, interpolation = cv2.INTER_NEAREST)
    
    return resizedIM

In [ ]:
# Read Color Image Dataset
DataPath = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\Original_Images\\"
path = DataPath
all_names = get_subfiles(path)
print("Number of Images:", len(all_names))

IMG = []
for i in range(len(all_names)):
    tmp = cv2.imread(path + all_names[i])
    IMG.append(tmp)

In [ ]:
# Color images
print('Color Images:')
Color_Image_List = IMG.copy()
Color_Image_Name = all_names

# create folder
color_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\Color_Images\\"
if not os.path.exists(color_images_path):
    os.makedirs(color_images_path)

# save color images
for i in range(len(Color_Image_Name)):
    print()
    print(Color_Image_Name[i])
    outpath = color_images_path + Color_Image_Name[i]
    cv2.imwrite(outpath, Color_Image_List[i])    
    ShowImage([Color_Image_List[i]], 1, 1)

In [ ]:
# Original images
print('Original Images:')
Original_Image_Name = all_names
Original_Image_List = []

# create folder
original_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\Original\\"
if not os.path.exists(original_images_path):
    os.makedirs(original_images_path)
    
# save images
for i in range(len(Original_Image_Name)):
    print()
    print(Original_Image_Name[i])
    image = Color_Image_List[i]
    tmp = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    Original_Image_List.append(tmp)
    outpath = original_images_path + Original_Image_Name[i]
    cv2.imwrite(outpath, Color_Image_List[i])    
    ShowImage([Original_Image_List[i]], 1, 1)


In [ ]:
# convert to grayscale and resize images function
def grayscale_resize (color_image_list, color_image_name, resize_para):
    print('Grayscale Resized Images ', resize_para)
    
    #create folder
    grayscaled_resized_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\Grayscaled_Resized_Images\\"
    if not os.path.exists(grayscaled_resized_images_path):
        os.makedirs(grayscaled_resized_images_path)
    
    Gray_Resized_Image_List = []
    Gray_Resized_Image_Name = color_image_name
    
    # save images
    for i in range(len(color_image_list)):
        resized_image = ResizeImage(color_image_list[i], DesiredWidth = resize_para, DesiredHeight = resize_para)
        gray_resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        Gray_Resized_Image_List.append(gray_resized_image)
        outpath = grayscaled_resized_images_path + Gray_Resized_Image_Name[i]
        cv2.imwrite(outpath, Gray_Resized_Image_List[i])    
        ShowImage([Gray_Resized_Image_List[i]], 1, 1)

    return Gray_Resized_Image_List, Gray_Resized_Image_Name

In [ ]:
# grayscaled resized 256x256 images
Gray_Resized_Image_List_256, Gray_Resized_Image_Name_256 = grayscale_resize(Color_Image_List, Color_Image_Name, 256)

In [5]:
# Create filters

# sobel 3x3
horizontal_sobel_3 = np.array((
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]),
    dtype="int")

vertical_sobel_3 = np.transpose(horizontal_sobel_3)
sobel_3 = [horizontal_sobel_3, vertical_sobel_3]

# sobel 5x5
horizontal_sobel_5 = np.array((
    [1, 4, 6, 4, 1],
    [2, 8, 12, 8, 2],
    [0, 0, 0, 0, 0],
    [-2, -8, -12, -8, -2],
    [-1, -4, -6, -4, -1]), 
    dtype="int")
vertical_sobel_5 = np.transpose(horizontal_sobel_5)
sobel_5 = [horizontal_sobel_5, vertical_sobel_5]

# Prewitt 3x3
horizontal_prewitt_3 = np.array((
    [1, 1, 1],
    [0, 0, 0],
    [-1, -1, -1]),
    dtype="int")
vertical_prewitt_3 = np.transpose(horizontal_prewitt_3)
prewitt_3 = [horizontal_prewitt_3, vertical_prewitt_3]

# Prewitt 5x5
horizontal_prewitt_5 = np.array((
    [1, 2, 3, 2, 1],
    [1, 2, 3, 2, 1],
    [0, 0, 0, 0, 0],
    [-1, -2, -3, -2, -1],
    [-1, -2, -3, -2, -1]), 
    dtype="int")
vertical_prewitt_5 = np.transpose(horizontal_prewitt_5)
prewitt_5 = [horizontal_prewitt_5, vertical_prewitt_5]


In [6]:
# detect edge using filter function
def detect_edge_using_filter(image_name, image_list, filter_type, folder_name="Edge_Images\\"):
    print('Detect Edge using Filter')
    
    # input threshold
    thresh = int(input("Input threshold: "))
    
    # create folder
    # edge images 
    edge_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\" + folder_name
    if not os.path.exists(edge_images_path):
        os.makedirs(edge_images_path)
    
    # Threshold Binary
    threshold_binary_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\" + folder_name + "Threshold_Binary\\"
    if not os.path.exists(threshold_binary_images_path):
        os.makedirs(threshold_binary_images_path)
    
    # Threshold Binary, Inverted 
    threshold_binary_inverted_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\" + folder_name + "Threshold_Binary_Inverted\\"
    if not os.path.exists(threshold_binary_inverted_images_path):
        os.makedirs(threshold_binary_inverted_images_path)        

    # Threshold Truncate 
    threshold_truncate_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\" + folder_name + "Threshold_Truncate\\"
    if not os.path.exists(threshold_truncate_images_path):
        os.makedirs(threshold_truncate_images_path)        
    
    # Threshold to zero 
    threshold_to_zero_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\" + folder_name + "Threshold_to_zero\\"
    if not os.path.exists(threshold_to_zero_images_path):
        os.makedirs(threshold_to_zero_images_path)        
    
    # Threshold to zero, inverted
    threshold_to_zero_inverted_images_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Image_Dataset\\" + folder_name + "Threshold_to_zero_Inverted\\"
    if not os.path.exists(threshold_to_zero_inverted_images_path):
        os.makedirs(threshold_to_zero_inverted_images_path)        
        
        
    Edge_Image_List = []
    Edge_Image_Name = image_name
      
    for i in range(len(Edge_Image_Name)):
        print(Edge_Image_Name[i])
        image = image_list[i]
        
        # filter
        horizontal_edge_image = cv2.filter2D(image, -1, filter_type[0])
        vertical_edge_image = cv2.filter2D(image, -1, filter_type[1])
        edge_image = horizontal_edge_image + vertical_edge_image
        
        Edge_Image_List.append(edge_image)
        
        # save edge images
        edge_outpath = edge_images_path + Edge_Image_Name[i]
        cv2.imwrite(edge_outpath, Edge_Image_List[i])    
        
        print()
        print('Image and Filtered Image:')
        ShowImage([image, edge_image], 1, 2)
        
        # threshold
        print()
        print('Thresholded Images')
        
        # threshold binary
        print('THRESH_BINARY')        
        ret1,thresh1 = cv2.threshold(edge_image,thresh,255,cv2.THRESH_BINARY)
        ShowImage([edge_image, thresh1], 1, 2)
        
        outpath1 = threshold_binary_images_path + Edge_Image_Name[i]
        cv2.imwrite(outpath1, thresh1)    
        

        # threshold binary, inverted
        print()
        print('THRESH_BINARY_INV')
        ret2,thresh2 = cv2.threshold(edge_image,thresh,255,cv2.THRESH_BINARY_INV)
        ShowImage([edge_image, thresh2], 1, 2)
        
        outpath2 = threshold_binary_inverted_images_path + Edge_Image_Name[i]
        cv2.imwrite(outpath2, thresh2)   
        
        # threshold truncate
        print()
        print('THRESH_TRUNC')
        ret3,thresh3 = cv2.threshold(edge_image,thresh,255,cv2.THRESH_TRUNC)
        ShowImage([edge_image, thresh3], 1, 2)
        
        outpath3 = threshold_truncate_images_path + Edge_Image_Name[i]
        cv2.imwrite(outpath3, thresh3)   
        
        # threshold to zero
        print()
        print('THRESH_TOZERO')
        ret4,thresh4 = cv2.threshold(edge_image,thresh,255,cv2.THRESH_TOZERO)
        ShowImage([edge_image, thresh4], 1, 2)
        
        outpath4 = threshold_to_zero_images_path + Edge_Image_Name[i]
        cv2.imwrite(outpath4, thresh4)   
        
        # threshold to zero, inverted
        print()
        print('THRESH_TOZERO_INV')
        ret5,thresh5 = cv2.threshold(edge_image,thresh,255,cv2.THRESH_TOZERO_INV)
        ShowImage([edge_image, thresh5], 1, 2)
        
        outpath5 = threshold_to_zero_inverted_images_path + Edge_Image_Name[i]
        cv2.imwrite(outpath5, thresh5)   
        
    return edge_image

In [ ]:
# Detect graycsale images 256x256 - Sobel 3x3
detect_edge_using_filter(Gray_Resized_Image_Name_256, Gray_Resized_Image_List_256, sobel_3, "Grayscaled_Resized_256_Images_Sobel_3\\")

In [ ]:
# Detect graycsale images 256x256 - Sobel 5x5
detect_edge_using_filter(Gray_Resized_Image_Name_256, Gray_Resized_Image_List_256, sobel_5, "Grayscaled_Resized_256_Images_Sobel_5\\")

In [ ]:
# Detect graycsale images 256x256 - Prewitt 3x3
detect_edge_using_filter(Gray_Resized_Image_Name_256, Gray_Resized_Image_List_256, prewitt_3, "Grayscaled_Resized_256_Images_Prewitt_3\\")

In [ ]:
# Detect graycsale images 256x256 - Prewitt 5x5
detect_edge_using_filter(Gray_Resized_Image_Name_256, Gray_Resized_Image_List_256, prewitt_5, "Grayscaled_Resized_256_Images_Prewitt_5\\")

In [ ]:
# Detect color images - Sobel 3x3
detect_edge_using_filter(Color_Image_Name, Color_Image_List, sobel_3, "Color_Edge_Images_Sobel_3\\")

In [ ]:
# Detect color images - Sobel 5x5
detect_edge_using_filter(Color_Image_Name, Color_Image_List, sobel_5, "Color_Edge_Images_Sobel_5\\")

In [ ]:
# Detect color images - Prewitt 3x3
detect_edge_using_filter(Color_Image_Name, Color_Image_List, prewitt_3, "Color_Edge_Images_Prewitt_3\\")

In [ ]:
# Detect color images - Prewitt 5x5
detect_edge_using_filter(Color_Image_Name, Color_Image_List, prewitt_5, "Color_Edge_Images_Prewitt_5\\")

In [7]:
# Read Video Dataset
VideoDataPath = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\"
video_path = VideoDataPath
video_all_names = get_subfiles(video_path)
print("Number of Videos:", len(video_all_names))
video_all_names

Number of Videos: 3


['0001.mp4', '0002.mp4', '0003.mp4']

In [8]:
# Detect video using filter
def detect_video_using_filter (original_file_name, detected_file_name, filter_type):
    import numpy as np
    import cv2
    
    cap = cv2.VideoCapture("C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\" + original_file_name)
    out = cv2.VideoWriter('C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\' + detected_file_name,cv2.VideoWriter_fourcc(*'H264'), cap.get(cv2.CAP_PROP_FPS) , (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))), True)

    while True:
        ret, frame = cap.read()

        if (ret==True):
            # filter
            horizontal_edge_frame = cv2.filter2D(frame, -1, filter_type[0])
            vertical_edge_frame = cv2.filter2D(frame, -1, filter_type[1])
            edge_frame = horizontal_edge_frame + vertical_edge_frame

            cv2.imshow("Detected", edge_frame)

            out.write(edge_frame)

            # Press Q on keyboard to stop recording
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Break the loop
        else:
            break 

    cap.release()

    out.release()

    cv2.destroyAllWindows()

In [9]:
# Detect video using Sobel 3x3

# create folder
sobel_3_video_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\Sobel_3_Videos\\"
if not os.path.exists(sobel_3_video_path):
    os.makedirs(sobel_3_video_path)

# detect
for i in range(len(video_all_names)):
    detect_video_using_filter (video_all_names[i], 'Sobel_3_' + video_all_names[i], sobel_3)

In [10]:
# Detect video using Sobel 5x5

# create folder
sobel_5_video_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\Sobel_5_Videos\\"
if not os.path.exists(sobel_5_video_path):
    os.makedirs(sobel_5_video_path)

# detect
for i in range(len(video_all_names)):
    detect_video_using_filter (video_all_names[i], 'Sobel_5_' + video_all_names[i], sobel_5)

In [11]:
# Detect video using Prewitt 3x3

# create folder
prewitt_3_video_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\Prewitt_3_Videos\\"
if not os.path.exists(prewitt_3_video_path):
    os.makedirs(prewitt_3_video_path)

# detect
for i in range(len(video_all_names)):
    detect_video_using_filter (video_all_names[i], 'Prewitt_3_' + video_all_names[i], prewitt_3)


In [ ]:
# Detect video using Prewitt 5x5

# create folder
prewitt_5_video_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\Prewitt_5_Videos\\"
if not os.path.exists(prewitt_5_video_path):
    os.makedirs(prewitt_5_video_path)

# detect
for i in range(len(video_all_names)):
    detect_video_using_filter (video_all_names[i], 'Prewitt_5_' + video_all_names[i], prewitt_5)


In [ ]:
# Detect webcam using filter function
def detect_webcam_using_filter (original_file_name, detected_file_name, filter_type):
    import numpy as np
    import cv2

    # Create a VideoCapture object
    cap = cv2.VideoCapture(0)

    # Create files
    out1 = cv2.VideoWriter('C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\' + original_file_name, cv2.VideoWriter_fourcc(*'H264'), cap.get(cv2.CAP_PROP_FPS) , (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))), True)
    out2 = cv2.VideoWriter('C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\' + detected_file_name, cv2.VideoWriter_fourcc(*'H264'), cap.get(cv2.CAP_PROP_FPS) , (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))), True)

    # Check if camera opened successfully
    if (cap.isOpened()==False):
        print("Unable to read camera feed!")

    while True:
        ret, frame = cap.read()

        if (ret==True):
            # filter
            horizontal_edge_frame = cv2.filter2D(frame, -1, filter_type[0])
            vertical_edge_frame = cv2.filter2D(frame, -1, filter_type[1])
            edge_frame = horizontal_edge_frame + vertical_edge_frame

            # Show        
            cv2.imshow('Stream', frame)
            cv2.imshow("Detected", edge_frame)

            # Save files
            out1.write(frame)
            out2.write(edge_frame)

            # Press Q on keyboard to stop recording
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Break the loop
        else:
            break 

    cap.release()

    out1.release()
    out2.release()

    cv2.destroyAllWindows()

In [ ]:
# Detect webcam using Sobel 3x3

# create folder
sobel_3_webcam_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\Sobel_3_Webcam\\"
if not os.path.exists(sobel_3_webcam_path):
    os.makedirs(sobel_3_webcam_path)

# detect
detect_webcam_using_filter ('Original_Video.mp4', 'Sobel_3_Detected_Video.mp4', sobel_3)

In [ ]:
# Detect webcam using Sobel 5x5

# create folder
sobel_5_webcam_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\Sobel_5_Webcam\\"
if not os.path.exists(sobel_5_webcam_path):
    os.makedirs(sobel_5_webcam_path)

# detect
detect_webcam_using_filter ('Original_Video.mp4', 'Sobel_5_Detected_Video.mp4', sobel_5)

In [ ]:
# Detect webcam using Prewitt 3x3

# create folder
prewitt_3_webcam_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\Prewitt_3_Webcam\\"
if not os.path.exists(prewitt_3_webcam_path):
    os.makedirs(prewitt_3_webcam_path)

# detect
detect_webcam_using_filter ('Original_Video.mp4', 'Prewitt_3_Detected_Video.mp4', prewitt_3)

In [ ]:
# Detect webcam using Sobel 5x5

# create folder
prewitt_5_webcam_path = "C:\\Users\\ADMIN\\Desktop\\NguyenNa\\Video_Dataset\\Prewitt_5_Webcam\\"
if not os.path.exists(prewitt_5_webcam_path):
    os.makedirs(prewitt_5_webcam_path)

# detect
detect_webcam_using_filter ('Original_Video.mp4', 'Prewitt_5_Detected_Video.mp4', prewitt_5)